<a href="https://colab.research.google.com/github/jrangelg/Artificial-intelligence/blob/main/Operanciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Investigación de operaciones

In [3]:
pip install pulp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 111.2 MB/s eta 0:00:00


In [4]:
import pulp

# Conjuntos y parámetros
refineries = ['Barrancabermeja', 'Cartagena', 'Orito']
countries = ['Estados Unidos', 'Japón', 'Reino Unido', 'Canadá']

costs = {
    'Barrancabermeja': [1500, 2200, 2000, 1800],
    'Cartagena': [1400, 2400, 1900, 2200],
    'Orito': [2000, 2500, 2100, 1900]
}

demands = {
    'Estados Unidos': 500000,
    'Japón': 300000,
    'Reino Unido': 200000,
    'Canadá': 250000
}

capacities = {
    'Barrancabermeja': 600000,
    'Cartagena': 500000,
    'Orito': 400000
}

# Inicializar el problema
prob = pulp.LpProblem("Optimización de distribución de petróleo", pulp.LpMinimize)

# Variables de decisión
x = {(r, p): pulp.LpVariable(f"X_{r}_{p}", lowBound=0, cat='Continuous') for r in refineries for p in countries}

# Función objetivo
prob += pulp.lpSum(costs[r][i] * x[r, c] for i, r in enumerate(refineries) for c in countries)

# Restricciones de capacidad de refinación
for r in refineries:
    prob += pulp.lpSum(x[r, c] for c in countries) <= capacities[r]

# Restricciones de demanda
for c in countries:
    prob += pulp.lpSum(x[r, c] for r in refineries) >= demands[c]

# Resolver el problema
prob.solve()

# Mostrar resultados
print("Estado:", pulp.LpStatus[prob.status])
print("Costo total mínimo: $", pulp.value(prob.objective))
print("\nCantidad de petróleo refinado a enviar (en litros):")
for r in refineries:
    for c in countries:
        print(f"{r} -> {c}: {x[r, c].varValue}")


Estado: Optimal
Costo total mínimo: $ 2340000000.0

Cantidad de petróleo refinado a enviar (en litros):
Barrancabermeja -> Estados Unidos: 500000.0
Barrancabermeja -> Japón: 0.0
Barrancabermeja -> Reino Unido: 0.0
Barrancabermeja -> Canadá: 100000.0
Cartagena -> Estados Unidos: 0.0
Cartagena -> Japón: 250000.0
Cartagena -> Reino Unido: 0.0
Cartagena -> Canadá: 0.0
Orito -> Estados Unidos: 0.0
Orito -> Japón: 50000.0
Orito -> Reino Unido: 200000.0
Orito -> Canadá: 150000.0


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [5]:
# Crear el problema dual
dual_prob = pulp.LpProblem("Problema Dual", pulp.LpMaximize)

# Variables duales asociadas a las restricciones de capacidad de refinación
dual_vars = {r: pulp.LpVariable(f"Y_{r}", lowBound=0, cat='Continuous') for r in refineries}

# Función objetivo dual
dual_prob += pulp.lpSum(capacities[r] * dual_vars[r] for r in refineries)

# Restricciones duales asociadas a la demanda de cada país
for c in countries:
    dual_prob += pulp.lpSum(costs[r][i] * dual_vars[r] for i, r in enumerate(refineries)) <= demands[c]

# Resolver el problema dual
dual_prob.solve()

# Mostrar resultados
print("\nValor óptimo del problema dual:", pulp.value(dual_prob.objective))



Valor óptimo del problema dual: 79999998.0
